In [1]:
import numpy as np
from collections import Counter

def lea_datos(archivo, i_clase=11, encabezado=True, delim=","):
    '''Una funcion para leer archivos con datos de clasificación. 
    Argumentos:
        archivo - direccion del archivo
        i_clase - indice de columna que contiene las clases. 
                  default es -1 y significa la ultima fila.
        header - si hay un encabezado
        delim - que separa los datos
    Regresa:
        Un tuple de los datos, clases y cabezazo en caso que hay.'''
    
    todo = np.loadtxt(archivo, dtype="S", delimiter=delim)    # para csv
    if(encabezado):
        encabezado = todo[0,:]
        todo = todo[1:,:]
    else: 
        encabezado = None
    
    clases = todo[:, i_clase]
    datos = np.delete(todo, [0, i_clase], axis=1)
    print ("Clases")
    for k,v in Counter(clases).items(): print (k,":",v)
    return (datos, clases, encabezado) 

In [2]:
datos, clases, encabezado = lea_datos("datos_peña.csv")  # _ significa que no nos interesa este valor 
clases.shape

Clases
b'1' : 9
b'0' : 14


(23,)

In [3]:
X = datos.astype("float")

In [4]:
X.shape

(23, 10)

In [5]:
#Data Imputation
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = -1.0, strategy = "most_frequent")
imp.fit_transform(X)


array([[  4.50000000e+00,   3.10000000e+01,   4.00000000e+00,
          5.00000000e+00,   1.12341354e+00,   9.40996006e-01,
          1.32221862e+00,   1.19122451e+00,   1.88000000e+02,
          5.00000000e-01],
       [  4.50000000e+00,   3.10000000e+01,   0.00000000e+00,
          6.00000000e+00,   1.64403306e+00,   1.09610206e+00,
          9.28685725e-01,   7.73842846e-01,   2.57000000e+02,
          4.00000000e-01],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          4.00000000e+00,   8.98748648e-01,   1.13970449e+00,
          1.14483608e+00,   1.23762166e+00,   2.17000000e+02,
          1.10000000e+00],
       [  2.95000000e+01,   7.90000000e+01,   0.00000000e+00,
          4.00000000e+00,   2.43689770e+00,   3.30602862e+00,
          8.07985056e-01,   4.15879549e-01,   2.77000000e+02,
          1.00000000e-01],
       [  0.00000000e+00,   2.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   2.25234594e+00,   1.66555157e+00,
          1.10416611e+00

In [6]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(oob_score=True)
clf.fit(X, clases)
print ("Exactitud estimado:", clf.oob_score_)

Exactitud estimado: 0.391304347826


/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:384: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [7]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt

cv = StratifiedKFold(y=clases, n_folds=2)
arboles_vals = np.arange(5,200,5)
busqueda = GridSearchCV(clf, param_grid=dict(n_estimators=arboles_vals), cv=cv)
busqueda.fit(X, clases)

print ('Mejor numero de arboles=',busqueda.best_params_,',exactitud =',busqueda.best_score_)

scores = [x[1] for x in busqueda.grid_scores_]

plt.plot(arboles_vals, scores)
plt.xlabel('Número de árboles')
plt.ylabel('exactitud cv')
plt.savefig('Grid_RF_treenumber.svg')
plt.show()


Mejor numero de arboles= {'n_estimators': 25} ,exactitud = 0.478260869565


/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:384: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:384: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [8]:
prof_vals = np.arange(1,12)
busqueda = GridSearchCV(clf, param_grid=dict(max_depth=prof_vals), cv=cv)
busqueda.fit(datos, clases)

print ('Mejor profundidad=',busqueda.best_params_,',exactitud =',busqueda.best_score_)

scores = [x[1] for x in busqueda.grid_scores_]
plt.plot(prof_vals, scores)
plt.xlabel('profundidad maxima')
plt.ylabel('exactitud cv')
plt.savefig('Grid_RF_depth.svg')
plt.show()


Mejor profundidad= {'max_depth': 8} ,exactitud = 0.521739130435


/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib/python3.4/site-packages/sklearn/ensemble/forest.py:384: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [9]:
#SVM
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score, StratifiedKFold

clf = SVC(kernel='rbf')
exas = cross_val_score(clf, X, clases, cv=2)    # escoge un numero adecuado de folds 
print ('cv exactitud =', exas.mean(), '±', exas.std())

cv exactitud = 0.609848484848 ± 0.0265151515152


In [10]:

from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt

cv = StratifiedKFold(y=clases, n_folds=3)     # usamos las primeras 100 muestras
C_rango = 10.0 ** np.arange(-5,5)
gamma_rango = 10.0 ** np.arange(-5,5)
busqueda = GridSearchCV(clf, param_grid=dict(C=C_rango, gamma=gamma_rango), cv=cv)
busqueda.fit(X, clases)        # usamos las primeras 100 muestras

print ('Mejor C, gamma =',busqueda.best_params_,',exactitud =',busqueda.best_score_)

scores = np.array( [x[1] for x in busqueda.grid_scores_] ) # Sacamos nada mas el segundo elemento de los scores
n_C, n_g = len(C_rango), len(gamma_rango)                  # sacamos el numero de parametros probados
scores = scores.reshape(n_C, n_g)                          # convertimos los scores en la cuadricula

# Ahora vamos a dibujar todo
plt.figure(figsize=(10,8))                     # hacemos el imagen un poco mas grande
plt.imshow(scores, interpolation='nearest')    # dibuja los scores representado por colores
plt.xlabel('gamma')                            # pon el titulo del eje x
plt.ylabel('C')                                # pon el titulo del eje y
plt.colorbar()                                 # pon un escala de colores para los scores
plt.xticks(np.arange(len(gamma_rango)), gamma_rango, rotation=45)    # pon los valores para gamma
plt.yticks(np.arange(len(C_rango)), C_rango)   # pon los valores para C
plt.savefig('heatmap_SVM.svg')
plt.show()


Mejor C, gamma = {'gamma': 0.001, 'C': 10.0} ,exactitud = 0.652173913043
